In [2]:
# !pip install -U tfx

In [3]:
import tensorflow as tf
from tfx import v1 as tfx

# TFX libraries
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# For performing feature selection and preprocessing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest, f_classif, chi2

# For feature visualization
import matplotlib.pyplot as plt 
import seaborn as sns

# Utilities
from tensorflow.python.lib.io import file_io
from tensorflow_metadata.proto.v0 import schema_pb2
from google.protobuf.json_format import MessageToDict
from  tfx.proto import example_gen_pb2
from tfx.types import standard_artifacts
from tensorflow_transform.tf_metadata import dataset_metadata, schema_utils
import tensorflow_transform.beam as tft_beam
import os
import pprint
import tempfile
import pandas as pd
from io import StringIO

# To ignore warnings from TF
tf.get_logger().setLevel('ERROR')

# For formatting print statements
pp = pprint.PrettyPrinter()

# Display versions of TF and TFX related packages
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))
print('TensorFlow Data Validation version: {}'.format(tfdv.__version__))
print('TensorFlow Transform version: {}'.format(tft.__version__))

TensorFlow version: 2.8.0
TFX version: 1.7.0
TensorFlow Data Validation version: 1.7.0
TensorFlow Transform version: 1.7.0


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
stroke_dataset = pd.read_csv('/content/drive/My Drive/Stroke Prediction ML System/healthcare-dataset-stroke-data.csv')
stroke_dataset.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [6]:
stroke_dataset.shape

(5110, 12)

In [7]:
stroke_dataset.isna().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [8]:
stroke_dataset = stroke_dataset.dropna()

In [9]:
stroke_dataset.groupby(['gender']).agg({'heart_disease': 'sum'})

,heart_disease
gender,
Female,100
Male,143
Other,0


In [10]:
stroke_dataset.groupby(['gender', 'stroke'])['stroke'].count()

gender  stroke
Female  0         2777
        1          120
Male    0         1922
        1           89
Other   0            1
Name: stroke, dtype: int64

In [11]:
stroke_dataset.dtypes

id                     int64
gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object

In [12]:
# Copy original dataset
stroke_dataset_num = stroke_dataset.copy()
stroke_dataset_cat = stroke_dataset.copy()

# Categorical columns
cat_columns = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

# Label column
label_column = ['stroke']

# Drop the categorical and label columns
stroke_dataset_num.drop(cat_columns+label_column+['id'], axis=1, inplace=True)

# Only get categorical columns
stroke_dataset_cat = stroke_dataset_cat[cat_columns]

# Preview the resuls
stroke_dataset_num.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi
0,67.0,0,1,228.69,36.6
2,80.0,0,1,105.92,32.5
3,49.0,0,0,171.23,34.4
4,79.0,1,0,174.12,24.0
5,81.0,0,0,186.21,29.0


In [13]:
# Preview the resuls
stroke_dataset_cat.head()

,gender,ever_married,work_type,Residence_type,smoking_status
0,Male,Yes,Private,Urban,formerly smoked
2,Male,Yes,Private,Rural,never smoked
3,Female,Yes,Private,Urban,smokes
4,Female,Yes,Self-employed,Rural,never smoked
5,Male,Yes,Private,Urban,formerly smoked


In [14]:
# Set the target values
y = stroke_dataset['stroke'].values

# Set the input values
X_num = stroke_dataset_num.values
X_cat = stroke_dataset_cat.values

In [15]:
# Create SelectKBest object using f_classif (ANOVA statistics) and chi2 (Chi-Squared) for 4 features
select_k_best_num = SelectKBest(f_classif, k=4)
select_k_best_cat = SelectKBest(chi2, k=4)

# Fit and transform the input data using select_k_best
X_new_num = select_k_best_num.fit_transform(X_num, y)
ord = OrdinalEncoder()
X_ohe = ord.fit_transform(X_cat)
X_new_cat = select_k_best_cat.fit_transform(X_ohe, y)

# Extract the features which are selected using get_support API
num_features_mask = select_k_best_num.get_support()
cat_features_mask = select_k_best_cat.get_support()

# Print the results
num_reqd_cols = pd.DataFrame({'Columns': stroke_dataset_num.columns, 'Retain': num_features_mask})
cat_reqd_cols = pd.DataFrame({'Columns': stroke_dataset_cat.columns, 'Retain': cat_features_mask})
reqd_cols = num_reqd_cols.append(cat_reqd_cols, ignore_index=True)

print(reqd_cols)

             Columns  Retain
0                age    True
1       hypertension    True
2      heart_disease    True
3  avg_glucose_level    True
4                bmi   False
5             gender    True
6       ever_married    True
7          work_type    True
8     Residence_type   False
9     smoking_status    True


In [16]:
# Location of the data and model directory
DATA_DIR = '/content/drive/My Drive/Stroke Prediction ML System'

# Set the paths to the reduced dataset
DATA_DIR_SELECT = f'{DATA_DIR}/select'
TRAINING_DATA_SELECT = f'{DATA_DIR_SELECT}/stroke_prediction_dataset.csv'

# # Create the directory
# !mkdir -p {TRAINING_DIR_FSELECT}

In [17]:
# Get the feature names from SelectKBest
num_feature_names = list(stroke_dataset_num.columns[num_features_mask])
cat_feature_names = list(stroke_dataset_cat.columns[cat_features_mask])

# Append the label columns
feature_names = num_feature_names + cat_feature_names + label_column

# Select the selected subset of columns
stroke_dataset_select = stroke_dataset[feature_names]

# Write CSV to the created directory
stroke_dataset_select.to_csv(TRAINING_DATA_SELECT, index=False)

# Preview the results
stroke_dataset_select.head()

,age,hypertension,heart_disease,avg_glucose_level,gender,ever_married,work_type,smoking_status,stroke
0,67.0,0,1,228.69,Male,Yes,Private,formerly smoked,1
2,80.0,0,1,105.92,Male,Yes,Private,never smoked,1
3,49.0,0,0,171.23,Female,Yes,Private,smokes,1
4,79.0,1,0,174.12,Female,Yes,Self-employed,never smoked,1
5,81.0,0,0,186.21,Male,Yes,Private,formerly smoked,1


In [18]:
# Location of the pipeline metadata store
PIPELINE_DIR = f'{DATA_DIR}/pipeline'

# Declare the InteractiveContext and use a local sqlite file as the metadata store.
context = InteractiveContext(pipeline_root=PIPELINE_DIR)

In [19]:
# Instantiate ExampleGen with the input CSV dataset
example_gen = tfx.components.CsvExampleGen(input_base=DATA_DIR_SELECT)

# Run the component using the InteractiveContext instance
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 33
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [20]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
    
# Run the component
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 34
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [21]:
# Display the results
context.show(statistics_gen.outputs['statistics'])

In [22]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'])

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 35
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [23]:
# Visualize the output
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',FLOAT,required,,-
'avg_glucose_level',FLOAT,required,,-
'ever_married',STRING,required,,'ever_married'
'gender',STRING,required,,'gender'
'heart_disease',INT,required,,-
'hypertension',INT,required,,-
'smoking_status',STRING,required,,'smoking_status'
'stroke',INT,required,,-
'work_type',STRING,required,,'work_type'


,Values
Domain,
'ever_married',"'No', 'Yes'"
'gender',"'Female', 'Male', 'Other'"
'smoking_status',"'Unknown', 'formerly smoked', 'never smoked', 'smokes'"
'work_type',"'Govt_job', 'Never_worked', 'Private', 'Self-employed', 'children'"


In [24]:
# Getting the schema uri
schema_uri = schema_gen.outputs['schema']._artifacts[0].uri

In [25]:
# Get the schema pbtxt file from the SchemaGen output
schema = tfdv.load_schema_text(os.path.join(schema_uri, 'schema.pbtxt'))

In [26]:
# Set the `age` feature to an appropriate range
tfdv.set_domain(schema, 'age', schema_pb2.FloatDomain(name='age', min=0, max=100))

# Set the `avg_glucose_level` feature to an appropriate range
tfdv.set_domain(schema, 'avg_glucose_level', schema_pb2.FloatDomain(name='avg_glucose_level', min=25.0, max=300.0))

# Set the `heart_disease`, `stroke`, and `hypertension` features to 0/1 and make it categorical
tfdv.set_domain(schema, 'heart_disease', schema_pb2.IntDomain(name='heart_disease', min=0, max=1, is_categorical=True))
tfdv.set_domain(schema, 'hypertension', schema_pb2.IntDomain(name='hypertension', min=0, max=1, is_categorical=True))
tfdv.set_domain(schema, 'stroke', schema_pb2.IntDomain(name='stroke', min=0, max=1, is_categorical=True))

tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'age',FLOAT,required,,min: 0.000000; max: 100.000000
'avg_glucose_level',FLOAT,required,,min: 25.000000; max: 300.000000
'ever_married',STRING,required,,'ever_married'
'gender',STRING,required,,'gender'
'heart_disease',INT,required,,min: 0; max: 1
'hypertension',INT,required,,min: 0; max: 1
'smoking_status',STRING,required,,'smoking_status'
'stroke',INT,required,,min: 0; max: 1
'work_type',STRING,required,,'work_type'


,Values
Domain,
'ever_married',"'No', 'Yes'"
'gender',"'Female', 'Male', 'Other'"
'smoking_status',"'Unknown', 'formerly smoked', 'never smoked', 'smokes'"
'work_type',"'Govt_job', 'Never_worked', 'Private', 'Self-employed', 'children'"


In [27]:
# Declare paths to the serving data
SERVING_DATA = f'{DATA_DIR}/serving/stoke_prediction_serving_dataset.csv'

In [28]:
# Read a subset of the training dataset
serving_data = pd.read_csv(TRAINING_DATA_SELECT, nrows=100)

# Drop the `Cover_Type` column
serving_data.drop(columns='stroke', inplace=True)

# Save the modified dataset
serving_data.to_csv(SERVING_DATA, index=False)

# Delete unneeded variable from memory
del serving_data

In [29]:
# Declare StatsOptions to use the curated schema
stats_options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)

# Compute the statistics of the serving dataset
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA, stats_options=stats_options)

# Detect anomalies in the serving dataset
anomalies = tfdv.validate_statistics(serving_stats, schema=schema)

# Display the anomalies detected
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'stroke',Column dropped,Column is completely missing


In [30]:
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

# Removing stroke feature from SERVING using TFDV
tfdv.get_feature(schema, 'stroke').not_in_environment.append('SERVING')

In [31]:
# Validate the serving dataset statistics in the `SERVING` environment
anomalies = tfdv.validate_statistics(serving_stats, schema=schema, environment='SERVING')

# Display the anomalies detected
tfdv.display_anomalies(anomalies)

In [32]:
# Declare the path to the updated schema directory
UPDATED_SCHEMA_DIR = f'{PIPELINE_DIR}/updated_schema'

# Declare the path to the schema file
schema_file = os.path.join(UPDATED_SCHEMA_DIR, 'schema.pbtxt')

# Save the curated schema to the said file
tfdv.write_schema_text(schema, schema_file)

In [33]:
# Load the schema from the directory we just created
new_schema = tfdv.load_schema_text(schema_file)

# Display the schema. Check that the Domain column still contains the ranges.
tfdv.display_schema(schema=new_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'age',FLOAT,required,,min: 0.000000; max: 100.000000
'avg_glucose_level',FLOAT,required,,min: 25.000000; max: 300.000000
'ever_married',STRING,required,,'ever_married'
'gender',STRING,required,,'gender'
'heart_disease',INT,required,,min: 0; max: 1
'hypertension',INT,required,,min: 0; max: 1
'smoking_status',STRING,required,,'smoking_status'
'stroke',INT,required,,min: 0; max: 1
'work_type',STRING,required,,'work_type'


,Values
Domain,
'ever_married',"'No', 'Yes'"
'gender',"'Female', 'Male', 'Other'"
'smoking_status',"'Unknown', 'formerly smoked', 'never smoked', 'smokes'"
'work_type',"'Govt_job', 'Never_worked', 'Private', 'Self-employed', 'children'"


In [34]:
new_schema.default_environment

['TRAINING', 'SERVING']

In [35]:
# Use ImportSchemaGen to put the curated schema to ML Metadata
user_schema_importer = tfx.components.ImportSchemaGen(schema_file=schema_file)

# Run the component
context.run(user_schema_importer, enable_cache=False)

context.show(user_schema_importer.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',FLOAT,required,,min: 0.000000; max: 100.000000
'avg_glucose_level',FLOAT,required,,min: 25.000000; max: 300.000000
'ever_married',STRING,required,,'ever_married'
'gender',STRING,required,,'gender'
'heart_disease',INT,required,,min: 0; max: 1
'hypertension',INT,required,,min: 0; max: 1
'smoking_status',STRING,required,,'smoking_status'
'stroke',INT,required,,min: 0; max: 1
'work_type',STRING,required,,'work_type'


,Values
Domain,
'ever_married',"'No', 'Yes'"
'gender',"'Female', 'Male', 'Other'"
'smoking_status',"'Unknown', 'formerly smoked', 'never smoked', 'smokes'"
'work_type',"'Govt_job', 'Never_worked', 'Private', 'Self-employed', 'children'"


In [36]:
# Use StatisticsGen to compute the statistics using the curated schema
statistics_gen_updated = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'],
    schema=user_schema_importer.outputs['schema'])

# Run the component
context.run(statistics_gen_updated)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 37
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [37]:
context.show(statistics_gen_updated.outputs['statistics'])

In [38]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen_updated.outputs['statistics'],
    schema=user_schema_importer.outputs['schema'])

# Run the component.
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 38
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [39]:
# Visualize the results
context.show(example_validator.outputs['anomalies'])

In [40]:
# Set the constants module filename
_stroke_constants_module_file = 'stroke_constants.py'

In [41]:
%%writefile {_stroke_constants_module_file}

SCALE_MINMAX_FEATURE_KEYS = ['age']

SCALE_Z_FEATURE_KEYS = ['avg_glucose_level']

VOCAB_FEATURE_KEYS = ['ever_married', 'gender', 'heart_disease', 'hypertension', 'work_type']

LABEL_KEY = 'stroke'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Writing stroke_constants.py


In [42]:
# Set the transform module filename
_stroke_transform_module_file = 'stroke_transform.py'

In [43]:
%%writefile {_stroke_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import stroke_constants

_SCALE_MINMAX_FEATURE_KEYS = stroke_constants.SCALE_MINMAX_FEATURE_KEYS
_SCALE_Z_FEATURE_KEYS = stroke_constants.SCALE_Z_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = stroke_constants.VOCAB_FEATURE_KEYS
_LABEL_KEY = stroke_constants.LABEL_KEY
_transformed_name = stroke_constants.transformed_name

def preprocessing_fn(inputs):

    features_dict = {}

    for feature in _SCALE_MINMAX_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transform using scaling of min_max function
        features_dict[_transformed_name(feature)] = tft.scale_by_min_max(data_col)

    for feature in _SCALE_Z_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transforming using scaling to z score
        features_dict[_transformed_name(feature)] = tft.scale_to_z_score(data_col)

    for feature in _VOCAB_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transforming using vocabulary available in column
        features_dict[_transformed_name(feature)] = tft.compute_and_apply_vocabulary(data_col)

    # No change in the label
    features_dict[_LABEL_KEY] = inputs[_LABEL_KEY]

    return features_dict


Writing stroke_transform.py


In [44]:
# Instantiate the Transform component
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=user_schema_importer.outputs['schema'],
    module_file=os.path.abspath(_stroke_transform_module_file))

# Run the component
context.run(transform, enable_cache=False)

ExecutionResult(
    component_id: Transform
    execution_id: 39
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [45]:
transform_uri = transform.outputs['transformed_examples'].get()[0].uri

In [46]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform_uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [50]:
# Get 3 records from the dataset and print the output to test

def get_records(dataset, num_records):
    """
    Extracts records from the given dataset.

    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    Returns:
        records: list of dictionaries of the row values
    """
    
    records = []
    
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))        
        # append to the records list
        records.append(example_dict)
      
    return records

transformed_dataset_subset = get_records(transformed_dataset, 3)
pp.pprint(transformed_dataset_subset)

[{'features': {'feature': {'age_xf': {'floatList': {'value': [0.59716797]}},
                           'avg_glucose_level_xf': {'floatList': {'value': [1.5289153]}},
                           'ever_married_xf': {'int64List': {'value': ['0']}},
                           'gender_xf': {'int64List': {'value': ['0']}},
                           'heart_disease_xf': {'int64List': {'value': ['0']}},
                           'hypertension_xf': {'int64List': {'value': ['0']}},
                           'stroke': {'int64List': {'value': ['1']}},
                           'work_type_xf': {'int64List': {'value': ['0']}}}}},
 {'features': {'feature': {'age_xf': {'floatList': {'value': [0.90234375]}},
                           'avg_glucose_level_xf': {'floatList': {'value': [-0.7955847]}},
                           'ever_married_xf': {'int64List': {'value': ['0']}},
                           'gender_xf': {'int64List': {'value': ['1']}},
                           'heart_disease_xf': {'int6